In [1]:
import json
import requests
import time

with open('./api_parsing/token.json') as f:
    data = json.load(f)

IP = data['ip']
TOKEN = data['access_token']

def get_user_friends(username):
    response = requests.get(f'https://api.jikan.moe/v4/users/{username}/friends')
    if response.status_code == 200:
        time.sleep(0.5)
        return response.json()
    else:
        print('REQUEST ERROR')
        time.sleep(1)
        return None
    
def get_user_list(username, params):
    global TOKEN

    headers = {'Authorization': f'Bearer {TOKEN}'}

    response = requests.get(f'https://api.myanimelist.net/v2/users/{username}/animelist', params=params, headers=headers)
    if response.status_code == 200:
        time.sleep(0.5)
        return response.json()
    else:
        print('REQUEST ERROR')
        time.sleep(1)
        return None

In [ ]:
from pymongo import MongoClient, errors

client = MongoClient("mongodb://localhost", 27017)

db = client['AnimeRecs']

col = db['UserNames']

try:
    col.create_index([('name', 1)], unique=True)
except errors.DuplicateKeyError as e:
    print(f"Error: {e}")

In [ ]:
#try insertion of name
def insertName(name):
    try:
        col.insert_one({'name': name})
    except errors.DuplicateKeyError:
        print(f'{name} already exists')

In [ ]:
#main script for data collection

#starting names (hardcoded)

starting_names = []